# Instrucciones

Construir una representación basada en Bag of Words. Se deberá leer el archivo de texto (ver adjunto) que contiene múltiples entradas de texto y dos clases distintas. El vector de características final debe tener como última columna el atributo de clase.
Saludos,

# Instalación necesaria

In [1]:
#%pip install spacy
#%pip install nltk
#%pip install sklearn
#%python -m spacy download es_core_news_sm
#%spacy download es_core_news_sm

#Si usas anaconda abre la terminal de tu Environments y ejecuta las lineas
#python -m spacy download es_core_news_sm
#spacy download es_core_news_sm
#Reiniciar terminal y continuar (ejecutando todo de nuevo)

# Importación de bibliotecas

In [2]:
import pandas as pd
import re
import spacy as spc
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

# cargar modelo de spaCy para español necesario para la lematizacion
nlp = spc.load("es_core_news_sm")

In [3]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jesus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Jesus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jesus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# Inicializar stopwords
spanish_stopwords = stopwords.words('spanish')

# Datos de entrada

In [5]:
# Cargar el archivo CSV
file_path = 'df_mini_HS.csv'
df = pd.read_csv(file_path)
print(df)

    label                                               text
0       1  ESAS COSAS Y OTRAS PUEDEN PASAR POR MANTENER A...
1       1  28: te amodio, odio a la perra de tu amiga ☺️☺...
2       1  @LaDivinaDiva Callate maldita perra. O seguro ...
3       1  @MarysabelPuerto Mejor callate cara de puta o ...
4       1  @xarita327 @TRIKYHUMOR @yonier2012 @casTa1326 ...
5       1              @CocotePR @ashleyhonohan callate puta
6       1  Y el inmigrante recibe ayuda del rico Estado l...
7       1  De los moros no se puede esperar nada bueno, y...
8       1  ¿Por que si a una mujer le pegan un tiro en la...
9       1  Analicemos esto: ¿Si te pones unos shorts así,...
10      0  "See... Tal vez les recordo como Peron protegi...
11      0  PIETRAPIERCE STORY: PURS SANGS ARABES STARS DE...
12      0  ¿Qué dice este de frivolizar el acoso escolar?...
13      0  #Suiza 🇨🇭 retira el permiso a 189 refugiados q...
14      0  Hoy quiero denunciaaaaaaar A LA GENTE PUTO GUA...
15      0  Redomicilie s

# Función de limpieza de oraciono

In [6]:
def limpiar_oraciono(oracion):
    emoji_pattern = re.compile("["
                                u"\U0001F600-\U0001F64F"  # Emoticonos
                                u"\U0001F300-\U0001F5FF"  # Símbolos y pictogramas
                                u"\U0001F680-\U0001F6FF"  # Transporte y símbolos
                                u"\U0001F700-\U0001F77F"  # Símbolos adicionales
                                u"\U0001F780-\U0001F7FF"  # Geometría adicional
                                u"\U0001F800-\U0001F8FF"  # Suplemento de pictogramas
                                u"\U0001F900-\U0001F9FF"  # Pictogramas adicionales
                                u"\U0001FA00-\U0001FA6F"  # Símbolos adicionales
                                u"\U00002702-\U000027B0"  # Otros símbolos (corazones, flechas)
                                u"\U000024C2-\U0001F251"  # Otros símbolos adicionales
                                "]+", flags=re.UNICODE)
    oracion = emoji_pattern.sub(r'', oracion)  # Eliminar emojis
    oracion = oracion.lower()
    oracion = re.sub(r"@\S+", "", oracion)  # Eliminar menciones a usuarios
    oracion = re.sub(r"http[s]?\://\S+", "", oracion)  # Eliminar enlaces 
    oracion = re.sub(r"#\S+", "", oracion)  # Eliminar hashtags
    oracion = re.sub(r"[0-9]", "", oracion)  # Eliminar números
    oracion = re.sub(r"(\(.*\))|(\[.*\])", "", oracion)  # Eliminar paréntesis y corchetes
    oracion = re.sub(r"\n", "", oracion)  # Eliminar caracteres de nueva línea
    oracion = re.sub(r"(http[s]?\://\S+)|([\[\(].*[\)\]])|([#@]\S+)|\n", "", oracion)  # Eliminar varios patrones
    oracion = re.sub(r"(\.)|(,)", "", oracion)  # Eliminar puntos y comas
    oracion = re.sub(r"[¡!]", "", oracion)  # Eliminar signos de admiración 
    oracion = re.sub(r"[¿?]", "", oracion)  # Eliminar signos de exclamación
    oracion = re.sub(r"[-\:\/\"\'\*\`\s]+", " ", oracion).strip() #Eliminar varios caracteres

    return oracion

# Función para tokenizar, remover stopwords y lematizar con spaCy

In [7]:
def procesamiento_oracion(oracion):
    oracion = limpiar_oraciono(oracion)
    tokens = word_tokenize(oracion)
    palabras_filtradas = [palabra for palabra in tokens if palabra not in spanish_stopwords]
    lema = nlp(" ".join(palabras_filtradas))
    oracion_procesada = " ".join([token.lemma_ for token in lema])

    return oracion_procesada

# Bag of Words

In [8]:
# Aplica la función de procesamiento a cada oración del archivo y crea una nueva columna "oracion_procesada"
df['oracion_procesada'] = df['text'].apply(procesamiento_oracion)

vectorizer = CountVectorizer()
vectores = vectorizer.fit_transform(df['oracion_procesada'])
vocabulario = vectorizer.get_feature_names_out()
print(f"Vocabulario:", vocabulario)

Vocabulario: ['acoso' 'agar' 'agresión' 'amigo' 'amodio' 'analicer' 'aprieto' 'arab'
 'arar' 'así' 'ayuda' 'bala' 'bastar' 'bueno' 'bus' 'cabeza' 'cadete'
 'callate' 'calle' 'camello' 'cara' 'categoria' 'cerebro' 'chavista'
 'chiste' 'chorizo' 'ciento' 'ciudadano' 'colaborador' 'colau' 'conto'
 'cortad' 'cosa' 'creer' 'cuatro' 'cuánto' 'cállate' 'decir' 'dejar'
 'denunciaaaaaaar' 'des' 'detras' 'dia' 'donbenitensar' 'durar' 'eichmann'
 'elite' 'emirato' 'encontrar' 'equipo' 'escolar' 'españa' 'esperar'
 'esperas' 'expulsar' 'falta' 'favor' 'femenino' 'festivit' 'frivolizar'
 'fuenlabrado' 'futbol' 'garcho' 'gente' 'guarra' 'hacer' 'hijo' 'hoy'
 'humillación' 'ilegal' 'inmigracion' 'inmigrante' 'inscrito'
 'josewifakers' 'juventud' 'kiev' 'ko' 'ladrón' 'llamado' 'luchado'
 'lugar' 'madre' 'maldito' 'mantener' 'mantero' 'marico' 'mata' 'medio'
 'mejor' 'metete' 'mientras' 'minato' 'morir' 'moro' 'mujer' 'mundo'
 'nacional' 'nazi' 'negrata' 'nota' 'odio' 'offshore' 'oler' 'olvidar'
 'on' 

In [9]:
# itaracion para todas las oraciones para imprimir resultados
for i, row in df.iterrows():
    oracion = row['text']
    oracion_lematizada = row['oracion_procesada']
    
    print(f"Oración de entrada [{i+1}]:", oracion)
    print(f"Oración lematizada [{i+1}]:", oracion_lematizada)
    print(f"Vectores Bag of Words [{i+1}]:", vectores[i].toarray())
    

Oración de entrada [1]: ESAS COSAS Y OTRAS PUEDEN PASAR POR MANTENER A LA INMIGRACION ILEGAL EN ESPAÑA Y NO EXPULSARLOS ¿NO? - La agresión de los manteros a un turista pone a Colau en un aprieto https://t.co/C7mZWXAl9P vía @Elperiodico
Oración lematizada [1]: cosa poder pasar mantener inmigracion ilegal españa expulsar él agresión mantero turista poner colau aprieto vía
Vectores Bag of Words [1]: [[0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0
  0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
  0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1]]
Oración de entrada [2]: 28: te amodio, odio a la perra de tu amiga ☺️☺️pero tú eres mi vida *tijerazo*
Oración lematizada [2]: amodio odio perra amigo vida tijerazo
Vectores Bag of Words [2]: [[0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [10]:
# Convertir la matriz BoW a un DataFrame y agregar la clase como última columna
df_bw = pd.DataFrame.sparse.from_spmatrix(vectores,columns = vocabulario)
df_bw['class'] = df['label'].values
df_bw.head()


,acoso,agar,agresión,amigo,amodio,analicer,aprieto,arab,arar,así,...,vez,vida,viola,voolka,vía,yogurín,you,árabe,él,class
0,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,1
1,0,0,0,1,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


# Conclusiones

Si bien realizar una bolsa de palabras (Bag of Words) es relativamente sencillo, la complejidad aumenta cuando los datos de entrada se vuelven menos comprensibles.

En esta tarea, por ejemplo, un filtro complejo y necesario sería eliminar todas las palabras que no pertenecen al idioma español. Sin embargo, si se hace utilizando un diccionario, esto podría eliminar palabras como "amodio", que aunque poco comunes, son correctas. Otro problema es el uso del español antiguo, el cual tendría que ser normalizado; por ejemplo, "cortad" debería transformarse en "cortar". Además, existen palabras sin sentido aparente que, en realidad, podrían ser simplemente errores tipográficos, y eliminarlas podría hacernos perder información valiosa. Esto nos llevaría a realizar una tarea similar a la de los correctores ortográficos como el de Google, donde corregiríamos errores y sugeriríamos alternativas con algo como "quizás quiso decir".

Sería bueno aclarar el propósito del Bag of Words y qué nivel de eliminación de palabras es adecuado para realizar una limpieza adecuada. También, sería útil conocer el objetivo de la tarea, de modo que el estudiante pueda decidir hasta qué punto llevar la limpieza 

## Algunas preguntas interesantes que se pueden investigar a futuro son:

1. ¿Realmente es necesario borrar los emojis, considerando que al final la Bag of Words no los toma en cuenta?
2. Otros caracteres que no se consideran en el proceso son: `-`, `/`, `:`, ``, `''`, `"`, `*`. ¿Es necesario filtrarlos?
3. ¿Por qué a veces la lematización cambia el género de las palabras, como en el caso de "puta" a "puto", mientras que en otras ocasiones sí respeta el género, dejando "puta"?